In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import seaborn as sns
from matplotlib import axes
import pandas as pd
import os
from os.path import join
import json

In [2]:

import sys, os
sys.path.append('/cluster/home/kamara/Explain')
from clutils.nbutils import *
os.getcwd()

'/cluster/home/kamara/Explain/notebooks'

In [3]:
def parseLogs(logfile, kw="rawlogs:", **kwargs):
    logs = {}
    dicts = []
    with open(logfile) as f:
        for i_line, line in enumerate(f):
            pos = line.find(kw)
            if pos >= 0:
                pos += len(kw)
                pos_tensor = line.find("tensor(")
                while pos_tensor >= 0:
                    comma_end = line.find(",", pos_tensor)
                    tensor_end = line.find(")", pos_tensor)
                    line_new = line[:pos_tensor] + line[pos_tensor + len("tensor("):comma_end] + line[tensor_end+1:]
                    print("LINE", line)
                    print("NEW", line_new)
                    if len(line_new) >= len(line):
                        print("No line reduction")
                        break
                    line = line_new
                    pos_tensor = line.find("tensor(")
                    
                #print(ast.literal_eval(line[pos:]))
                #line[pos:] = line[pos:].replace("'", "")
                kline = line[pos:]
                kline = kline.replace("true", "True")
                try:
                    dic = ast.literal_eval(kline)
                except:
                    # print("Encountering weird patterns in logs")
                    # print("Line number %d" % i_line)
                    # print(line)
                    line = line.replace("nan,", "-1e8,")
                    line = line.replace("NaN", "-1e8")
                    try:
                        dic = ast.literal_eval(line[pos:])
                    except:
                        print("Unable to replace NaNs")
                        print(line)
                        continue
                
                for k in dic.keys():
                    if k not in logs:
                        logs[k] = []
                    logs[k].append(dic[k])
                dicts.append(dic)

    df = pd.DataFrame(dicts)

    return logs, df


In [4]:
def get_info_real(logdir):
    infos, fidelity, initial_mask_infos, transformed_mask_infos = {}, {}, {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__infos:')
            _, initial_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__initial_edge_mask_infos:')
            _, transformed_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__transformed_mask_infos:')
            _, fidelity[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__fidelity:')             
    return(infos, initial_mask_infos, transformed_mask_infos, fidelity)

def get_info_syn(logdir):
    infos, fidelity, accuracy, initial_mask_infos, transformed_mask_infos = {}, {}, {}, {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__infos:')
            _, initial_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__initial_edge_mask_infos:')
            _, transformed_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__transformed_mask_infos:')
            _, accuracy[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__accuracy:') 
            _, fidelity[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__fidelity:') 
    return(infos, initial_mask_infos, transformed_mask_infos, accuracy, fidelity)



def get_gnn_info(logdir):
    gnn_train, gnn_test = {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, gnn_train[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__gnn_train_scores: ')
            _, gnn_test[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__gnn_test_scores: ')
    return(gnn_train, gnn_test)


def get_param_ranges(jsonpath, dataset = ""):
    f = open(jsonpath,)
    data = json.load(f)
    ranges = data['params']
    ranges['none'] = ''
    return(ranges)   

In [5]:
def get_df_results(logs, ranges, name, metrics, selection = 'last'):
    dicts = []
    for params_set in enumerateParams(ranges):
        key = name.format(**params_set)
        # if key in logs and len(logs[key]) >= 1 and metric in logs[key]:
        any_metric = (key in logs) and (len(logs[key]) >= 1) and any([metric in logs[key] for metric in metrics])
        if any_metric:
            if selection == 'all':
                for index, row in logs[key].iterrows():
                    metrics_dict = {
                        metric: select_value(row.to_frame().T, metric, 'last') if key in logs and len(logs[key]) >= 1 and metric in logs[key] else -1
                        for metric in metrics
                    }
                    dicts.append(dictmerge(params_set, metrics_dict))
                    
            else:
                metrics_dict = {
                    metric: select_value(logs[key], metric, selection) if key in logs and len(logs[key]) >= 1 and metric in logs[key] else -1
                    for metric in metrics
                }
                dicts.append(dictmerge(params_set, metrics_dict))

    df = pd.DataFrame(dicts)
    #df = df.drop(columns=['none'])
    return df

## Node Classification - Real world

### GNN scores

In [28]:
df_gnn_train = get_df_results(gnn_train, ranges, name, metrics = ["prec", "recall", "f1_score", "acc"])
df_gnn_test = get_df_results(gnn_test, ranges, name, metrics = ["prec", "recall", "f1_score", "acc"])

df_gnn_train = df_gnn_train.groupby(by=['dataset']).mean().reset_index()
df_gnn_test = df_gnn_test.groupby(by=['dataset']).mean().reset_index()

gnn_scores = pd.merge(df_gnn_train, df_gnn_test, on=['dataset', 'sparsity', 'num_test'], suffixes=['_train', '_test'])
gnn_scores = gnn_scores.drop(columns=["sparsity", "num_test", "seed_train", "seed_test"])
gnn_scores

,dataset,prec_train,recall_train,acc_train,prec_test,recall_test,acc_test
0,actor,0.594775,0.480935,0.538197,0.260749,0.249035,0.285942
1,chameleon,0.817928,0.806324,0.806410,0.646635,0.634489,0.632237
2,citeseer,1.000000,1.000000,1.000000,0.651307,0.652012,0.676400
3,cora,1.000000,1.000000,1.000000,0.781336,0.817381,0.803496
4,cornell,0.927170,0.908066,0.974694,0.354803,0.338859,0.532487
5,facebook,0.931203,0.926196,0.933334,0.923301,0.917619,0.926356
6,pubmed,1.000000,1.000000,1.000000,0.771646,0.781766,0.778500
7,texas,0.972625,0.966848,0.991954,0.296897,0.277057,0.510811
8,wisconsin,0.980174,0.955045,0.975833,0.397882,0.406152,0.535294


In [29]:
#gnn_scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_gnn_scores.csv', index=False)

### Explainability methods scores

In [58]:
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges", "mask_sparsity_init", "non_zero_values_init"])
df_initial_mask_infos = get_df_results(initial_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg"])
df_tranformed_mask_infos = get_df_results(transformed_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg"])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 
                                                           'fidelity_gnn_acc+', 'fidelity_gnn_acc-', 'fidelity_gnn_prob+', 'fidelity_gnn_prob-'])

In [31]:
df_mask_infos = pd.merge(df_initial_mask_infos, df_tranformed_mask_infos, suffixes=['_init', '_transf'], on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "sparsity", "true_label_as_target", "hard_mask"])
scores = df_fid
scores = pd.merge(scores, df_mask_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "sparsity", "true_label_as_target", "hard_mask"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "sparsity", "true_label_as_target", "hard_mask"])
scores
#scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_sparsity_all.csv', index=False)


,none_x,explainer_name,sparsity,dataset,true_label_as_target,hard_mask,explain_graph_x,num_test,seed,data_save_dir,...,explain_graph_transf,mask_size_transf,mask_entropy_transf,max_avg_transf,none_y,explain_graph_y,time,number_of_edges,mask_sparsity_init,non_zero_values_init
0,,random,0.00,cora,True,True,False,100,0,data,...,False,13249.92,9.299591,0.428312,,False,0.0002,13264,0.001062,13249.92
1,,random,0.00,cora,True,True,False,100,1,data,...,False,13250.48,9.299754,0.490545,,False,0.0002,13264,0.001019,13250.48
2,,random,0.00,cora,True,True,False,100,2,data,...,False,13250.34,9.299545,0.498040,,False,0.0002,13264,0.001030,13250.34
3,,random,0.00,cora,True,True,False,100,3,data,...,False,13249.36,9.299762,0.489739,,False,0.0002,13264,0.001104,13249.36
4,,random,0.00,cora,True,True,False,100,4,data,...,False,13249.54,9.299681,0.470068,,False,0.0002,13264,0.001090,13249.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12327,,pgmexplainer,0.99,facebook,False,True,False,100,1,data,...,False,2768.48,7.475550,0.516261,,False,89.3687,364116,0.944714,20130.39
12328,,pgmexplainer,0.99,facebook,False,True,False,100,2,data,...,False,2392.10,7.094828,0.522513,,False,87.7417,364116,0.948680,18686.40
12329,,pgmexplainer,0.99,facebook,False,True,False,100,3,data,...,False,2672.38,7.429812,0.492495,,False,91.9752,364116,0.957629,15427.86
12330,,pgmexplainer,0.99,facebook,False,True,False,100,4,data,...,False,2674.88,7.356349,0.536693,,False,87.2857,364116,0.951040,17826.97


In [ ]:
df_nan = scores[scores.isna().any(axis=1)]


In [32]:
res = scores.groupby(by=['dataset', "true_label_as_target", "hard_mask", 'sparsity', 'explainer_name']).mean()
res = res.drop(columns=['seed'])
#res[['mask_size', "mask_entropy", "max_avg"]]
res

num_test  \
dataset   true_label_as_target hard_mask sparsity explainer_name                 
actor     False                False     0.000    basic_gnnexplainer     100.0   
                                                  distance               100.0   
                                                  gnnexplainer           100.0   
                                                  gradcam                100.0   
                                                  ig                     100.0   
...                                                                        ...   
wisconsin True                 True      0.997    gnnexplainer           100.0   
                                                  ig                     100.0   
                                                  occlusion              100.0   
                                                  pagerank               100.0   
                                                  random                 100.0   

                                                                      fidelity_acc+  \
dataset   true_label_as_target hard_mask sparsity explainer_name                      
actor     False                False     0.000    basic_gnnexplainer         -1.000   
                                                  distance                   -1.000   
                                                  gnnexplainer               -1.000   
                                                  gradcam                    -1.000   
                                                  ig                         -1.000   
...                                                                             ...   
wisconsin True                 True      0.997    gnnexplainer                0.666   
                                                  ig                          0.366   
                                                  occlusion                   0.667   
                                                  pagerank                    0.290   
                                                  random                      0.228   

                                                                      fidelity_acc-  \
dataset   true_label_as_target hard_mask sparsity explainer_name                      
actor     False                False     0.000    basic_gnnexplainer         -1.000   
                                                  distance                   -1.000   
                                                  gnnexplainer               -1.000   
                                                  gradcam                    -1.000   
                                                  ig                         -1.000   
...                                                                             ...   
wisconsin True                 True      0.997    gnnexplainer                0.458   
                                                  ig                          0.063   
                                                  occlusion                   0.205   
                                                  pagerank                    0.077   
                                                  random                      0.804   

                                                                      fidelity_prob+  \
dataset   true_label_as_target hard_mask sparsity explainer_name                       
actor     False                False     0.000    basic_gnnexplainer       -1.000000   
                                                  distance                 -1.000000   
                                                  gnnexplainer             -1.000000   
                                                  gradcam                  -1.000000   
                                                  ig                       -1.000000   
...                                                                              ...   
wisconsin True                 True

In [33]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_sparsity_avg10expe.csv', index=False)

## Topk

In [6]:
def get_info(logdir):
    infos, fidelity, initial_mask_infos, transformed_mask_infos= {}, {}, {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__infos:')
            _, initial_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__initial_edge_mask_infos:')
            _, transformed_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__transformed_mask_infos:')
            _, fidelity[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__fidelity:')             
    return(infos, initial_mask_infos, transformed_mask_infos, fidelity)



In [7]:
logdir = f'/cluster/work/zhang/kamara/checkpoints/node_classification/real/topk/topk_real_5/logs'
jsonpath = f'/cluster/home/kamara/Explain/configs/real/topk/config_real_topk_5expe.json'
infos, initial_mask_infos, transformed_mask_infos, fidelity = get_info(logdir)
gnn_train, gnn_test = get_gnn_info(logdir)

print(list(fidelity.keys())[1])
ranges = get_param_ranges(jsonpath)

_explainer_name=gradcam_dataset=squirrel_true_label_as_target=False_hard_mask=True_seed=1


In [8]:
#name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_dataset={dataset}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}'
name = '{none}_explainer_name={explainer_name}_dataset={dataset}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}_seed={seed}'


In [9]:
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges"])#, "edge_mask_sparsity_init", "edge_mask_size_init", "edge_mask_connected_init"])
df_initial_mask_infos = get_df_results(initial_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg"])
df_transformed_mask_infos = get_df_results(transformed_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg", "cc_ratio", "topk"], selection="all")

df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 
                                                           'fidelity_gnn_acc+', 'fidelity_gnn_acc-', 'fidelity_gnn_prob+', 'fidelity_gnn_prob-', 'topk'], selection="all")

In [11]:
df_mask_infos = pd.merge(df_initial_mask_infos, df_transformed_mask_infos, suffixes=['_init', '_transf'], on=["dataset", "explainer_name", "data_save_dir", "seed", "true_label_as_target", "hard_mask"])
scores = df_fid
scores = pd.merge(scores, df_mask_infos, on=["dataset", "explainer_name", "data_save_dir", "seed", "true_label_as_target", "hard_mask", "topk"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "data_save_dir", "seed", "true_label_as_target", "hard_mask"])
scores
#scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_sparsity_all.csv', index=False)


,none_x,explainer_name,dataset,params_list_x,strategy_x,true_label_as_target,hard_mask,explain_graph_x,num_test_x,seed,...,mask_entropy_transf,max_avg_transf,cc_ratio,none_y,params_list_y,strategy_y,explain_graph_y,num_test_y,time,number_of_edges
0,,random,cora,"1,5,10,15,20,25,50,100",topk,True,True,False,100,0,...,0.000000,1.000000,0.590000,,"1,5,10,15,20,25,50,100",topk,False,100,0.0002,13264
1,,random,cora,"1,5,10,15,20,25,50,100",topk,True,True,False,100,0,...,1.609438,0.999720,0.556905,,"1,5,10,15,20,25,50,100",topk,False,100,0.0002,13264
2,,random,cora,"1,5,10,15,20,25,50,100",topk,True,True,False,100,0,...,2.302585,0.999468,0.553509,,"1,5,10,15,20,25,50,100",topk,False,100,0.0002,13264
3,,random,cora,"1,5,10,15,20,25,50,100",topk,True,True,False,100,0,...,2.708050,0.999279,0.551098,,"1,5,10,15,20,25,50,100",topk,False,100,0.0002,13264
4,,random,cora,"1,5,10,15,20,25,50,100",topk,True,True,False,100,0,...,2.995732,0.999034,0.548972,,"1,5,10,15,20,25,50,100",topk,False,100,0.0002,13264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19845,,subgraphx,squirrel,"1,5,10,15,20,25,50,100",topk,False,True,False,100,3,...,0.000000,1.000000,1.000000,,"1,5,10,15,20,25,50,100",topk,False,100,36816.8081,401907
19846,,subgraphx,squirrel,"1,5,10,15,20,25,50,100",topk,False,True,False,100,3,...,0.000000,1.000000,1.000000,,"1,5,10,15,20,25,50,100",topk,False,100,36816.8081,401907
19847,,subgraphx,squirrel,"1,5,10,15,20,25,50,100",topk,False,True,False,100,3,...,0.000000,1.000000,1.000000,,"1,5,10,15,20,25,50,100",topk,False,100,36816.8081,401907
19848,,subgraphx,squirrel,"1,5,10,15,20,25,50,100",topk,False,True,False,100,3,...,0.000000,1.000000,1.000000,,"1,5,10,15,20,25,50,100",topk,False,100,36816.8081,401907


In [12]:
res = scores.groupby(by=['dataset', "true_label_as_target", "hard_mask", 'topk', 'explainer_name']).apply(lambda x: x.mean(skipna=True))
res = res.drop(columns=['seed', 'topk'])
#res[['mask_size', "mask_entropy", "max_avg"]]
res

/tmp/ipykernel_18344/2420268559.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  res = scores.groupby(by=['dataset', "true_label_as_target", "hard_mask", 'topk', 'explainer_name']).apply(lambda x: x.mean(skipna=True))


num_test_x  \
dataset   true_label_as_target hard_mask topk  explainer_name                   
actor     False                False     1.0   basic_gnnexplainer       100.0   
                                               distance                 100.0   
                                               gnnexplainer             100.0   
                                               gradcam                  100.0   
                                               ig                       100.0   
...                                                                       ...   
wisconsin True                 True      100.0 pgexplainer              100.0   
                                               pgmexplainer             100.0   
                                               random                   100.0   
                                               sa                       100.0   
                                               subgraphx                100.0   

                                                                   fidelity_acc+  \
dataset   true_label_as_target hard_mask topk  explainer_name                      
actor     False                False     1.0   basic_gnnexplainer         -1.000   
                                               distance                   -1.000   
                                               gnnexplainer               -1.000   
                                               gradcam                    -1.000   
                                               ig                         -1.000   
...                                                                          ...   
wisconsin True                 True      100.0 pgexplainer                 0.160   
                                               pgmexplainer                0.448   
                                               random                      0.136   
                                               sa                          0.542   
                                               subgraphx                   0.160   

                                                                   fidelity_acc-  \
dataset   true_label_as_target hard_mask topk  explainer_name                      
actor     False                False     1.0   basic_gnnexplainer         -1.000   
                                               distance                   -1.000   
                                               gnnexplainer               -1.000   
                                               gradcam                    -1.000   
                                               ig                         -1.000   
...                                                                          ...   
wisconsin True                 True      100.0 pgexplainer                 0.340   
                                               pgmexplainer                0.094   
                                               random                      0.424   
                                               sa                          0.382   
                                               subgraphx                   0.020   

                                                                   fidelity_prob+  \
dataset   true_label_as_target hard_mask topk  explainer_name                       
actor     False                False     1.0   basic_gnnexplainer       -1.000000   
                                               distance                 -1.000000   
                                               gnnexplainer             -1.000000   
                                               gradcam                  -1.000000   
                                               ig                       -1.000000   
...                                                                           ...   
wisconsin True                 True      100.0 pgexplainer               0.016246   
                                               pgmexplainer   

In [13]:
res.columns

Index(['num_test_x', 'fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+',
       'fidelity_prob-', 'fidelity_gnn_acc+', 'fidelity_gnn_acc-',
       'fidelity_gnn_prob+', 'fidelity_gnn_prob-', 'num_test_init',
       'mask_size_init', 'mask_entropy_init', 'max_avg_init',
       'num_test_transf', 'mask_size_transf', 'mask_entropy_transf',
       'max_avg_transf', 'cc_ratio', 'num_test_y', 'time', 'number_of_edges'],
      dtype='object')

In [14]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/topk/nc_real_topk_5expe.csv', index=False)

## Test Prediction Type: wrong or correct

In [6]:
def get_info(logdir):
    infos, fidelity, initial_mask_infos, transformed_mask_infos = {}, {}, {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__infos:')
            _, initial_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__initial_edge_mask_infos:')
            _, transformed_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__transformed_mask_infos:')
            _, fidelity[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__fidelity:')             
    return(infos, initial_mask_infos, transformed_mask_infos, fidelity)



In [26]:
logdir = f'/cluster/work/zhang/kamara/checkpoints/node_classification/real/prediction_type/pred_type_real_1/logs'
jsonpath = f'/cluster/home/kamara/Explain/configs/real/config_real_1expe_testpred.json'
infos, initial_mask_infos, transformed_mask_infos, fidelity = get_info(logdir)
gnn_train, gnn_test = get_gnn_info(logdir)

print(list(fidelity.keys())[1])
ranges = get_param_ranges(jsonpath)

_explainer_name=gradcam_dataset=facebook_testing_pred=wrong_true_label_as_target=True_hard_mask=True


In [27]:
#name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_dataset={dataset}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}'
name = '{none}_explainer_name={explainer_name}_dataset={dataset}_testing_pred={testing_pred}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}'


In [28]:
df_infos = get_df_results(infos, ranges, name, metrics = ['time', 'number_of_edges'])
df_initial_mask_infos = get_df_results(initial_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg"])
df_tranformed_mask_infos = get_df_results(transformed_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg", "topk"], selection="all")
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 
                                                           'fidelity_gnn_acc+', 'fidelity_gnn_acc-', 'fidelity_gnn_prob+', 'fidelity_gnn_prob-', 'topk'], selection="all")

In [30]:
df_mask_infos = pd.merge(df_initial_mask_infos, df_tranformed_mask_infos, suffixes=['_init', '_transf'], on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask", "testing_pred"])
scores = df_fid
scores = pd.merge(scores, df_mask_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask", "topk", "testing_pred"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask", "testing_pred"])
scores
#scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_sparsity_all.csv', index=False)


,none_x,explainer_name,params_list_x,strategy_x,dataset,testing_pred,true_label_as_target,hard_mask,explain_graph_x,num_test,...,explain_graph_transf,mask_size_transf,mask_entropy_transf,max_avg_transf,none_y,params_list_y,strategy_y,explain_graph_y,time,number_of_edges
0,,random,"1,2,5,10,20,100",topk,cora,correct,True,True,False,50,...,False,1.000000,0.000000,1.000000,,"1,2,5,10,20,100",topk,False,0.0002,13264
1,,random,"1,2,5,10,20,100",topk,cora,correct,True,True,False,50,...,False,2.000000,0.693147,0.999929,,"1,2,5,10,20,100",topk,False,0.0002,13264
2,,random,"1,2,5,10,20,100",topk,cora,correct,True,True,False,50,...,False,5.000000,1.609438,0.999720,,"1,2,5,10,20,100",topk,False,0.0002,13264
3,,random,"1,2,5,10,20,100",topk,cora,correct,True,True,False,50,...,False,10.000000,2.302585,0.999468,,"1,2,5,10,20,100",topk,False,0.0002,13264
4,,random,"1,2,5,10,20,100",topk,cora,correct,True,True,False,50,...,False,20.000000,2.995732,0.999034,,"1,2,5,10,20,100",topk,False,0.0002,13264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3839,,subgraphx,"1,2,5,10,20,100",topk,chameleon,wrong,True,True,False,50,...,False,1.428571,0.297063,1.000000,,"1,2,5,10,20,100",topk,False,5117.4368,65019
3840,,subgraphx,"1,2,5,10,20,100",topk,chameleon,wrong,True,True,False,50,...,False,2.571429,0.657881,1.000000,,"1,2,5,10,20,100",topk,False,5117.4368,65019
3841,,subgraphx,"1,2,5,10,20,100",topk,chameleon,wrong,True,True,False,50,...,False,3.142857,0.754016,1.000000,,"1,2,5,10,20,100",topk,False,5117.4368,65019
3842,,subgraphx,"1,2,5,10,20,100",topk,chameleon,wrong,True,True,False,50,...,False,3.142857,0.754016,1.000000,,"1,2,5,10,20,100",topk,False,5117.4368,65019


In [31]:
res = scores.groupby(by=['dataset', "true_label_as_target", "hard_mask", 'topk', 'explainer_name', 'testing_pred']).mean()
res = res.drop(columns=['seed'])
#res[['mask_size', "mask_entropy", "max_avg"]]
res

num_test  \
dataset  true_label_as_target hard_mask topk  explainer_name     testing_pred             
actor    False                False     1.0   basic_gnnexplainer correct           50.0   
                                                                 wrong             50.0   
                                              distance           correct           50.0   
                                                                 wrong             50.0   
                                              gnnexplainer       correct           50.0   
...                                                                                 ...   
facebook True                 True      100.0 occlusion          correct           50.0   
                                                                 wrong             50.0   
                                              pgmexplainer       wrong             50.0   
                                              sa                 correct           50.0   
                                                                 wrong             50.0   

                                                                               fidelity_acc+  \
dataset  true_label_as_target hard_mask topk  explainer_name     testing_pred                  
actor    False                False     1.0   basic_gnnexplainer correct               -1.00   
                                                                 wrong                 -1.00   
                                              distance           correct               -1.00   
                                                                 wrong                 -1.00   
                                              gnnexplainer       correct               -1.00   
...                                                                                      ...   
facebook True                 True      100.0 occlusion          correct                0.81   
                                                                 wrong                  0.11   
                                              pgmexplainer       wrong                  0.11   
                                              sa                 correct                0.86   
                                                                 wrong                  0.12   

                                                                               fidelity_acc-  \
dataset  true_label_as_target hard_mask topk  explainer_name     testing_pred                  
actor    False                False     1.0   basic_gnnexplainer correct               -1.00   
                                                                 wrong                 -1.00   
                                              distance           correct               -1.00   
                                                                 wrong                 -1.00   
                                              gnnexplainer       correct               -1.00   
...                                                                                      ...   
facebook True                 True      100.0 occlusion          correct                0.09   
                                                                 wrong                  0.24   
                                              pgmexplainer       wrong                  0.24   
                                              sa                 correct                0.50   
                                                                 wrong                  0.46   

                                                                               fidelity_prob+  \
dataset  true_label_as_target hard_mask topk  explainer_name     testing_pred                   
actor    False                False     1.0   basic_gnnexplainer correct            -1.000000   
                                                                 wrong              -1.000000   
         

In [33]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/prediction_type/nc_real_predtype_1expe.csv', index=False)

## Node Classification - Real world - 10 seeds

In [6]:
datasets = [
      "cora",
      "pubmed",
      "citeseer",
      "cornell",
      "texas",
      "wisconsin",
      "actor",
      "chameleon",
    "squirrel",
    "facebook"
    ]

### Sparsity

In [7]:
#name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_dataset={dataset}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}'
#name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_dataset={dataset}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}_seed={seed}'
name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}_seed={seed}'
#name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}'


In [8]:
versions = ["v0", "v1", "v2", "v3", "v4", "v5"]
SCORES = []
for v in version:

    logdir = f'/cluster/work/zhang/kamara/checkpoints/node_classification/real/topk/topk_real_5_{data}/logs'
    jsonpath = f'/cluster/home/kamara/Explain/configs/real/topk/config_real_topk_5expe_{data}.json'
    ranges = get_param_ranges(jsonpath)
    infos, initial_mask_infos, transformed_mask_infos, fidelity = get_info(logdir)
    
    df_infos = get_df_results(infos, ranges, name, metrics = ['dataset', 'time', "number_of_edges", "mask_sparsity_init", "non_zero_values_init"])
    print(df_infos)
    df_initial_mask_infos = get_df_results(initial_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg"])
    df_tranformed_mask_infos = get_df_results(transformed_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg"])
    df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 
                                                           'fidelity_gnn_acc+', 'fidelity_gnn_acc-', 'fidelity_gnn_prob+', 'fidelity_gnn_prob-',
                                                           'mask_sparsity', 'expl_edges'])
    
    df_mask_infos = pd.merge(df_initial_mask_infos, df_tranformed_mask_infos, suffixes=['_init', '_transf'], on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "sparsity", "true_label_as_target", "hard_mask"])
    scores = df_fid
    scores = pd.merge(scores, df_mask_infos, on=["explainer_name", "num_test", "data_save_dir", "seed", "sparsity", "true_label_as_target", "hard_mask"])
    scores = pd.merge(scores, df_infos, on=["explainer_name", "num_test", "data_save_dir", "seed", "sparsity", "true_label_as_target", "hard_mask"])
    res = scores.groupby(by=['dataset', "true_label_as_target", "hard_mask", 'sparsity', 'explainer_name']).mean()
    res = res.drop(columns=['seed'])
    print('res', res)

    SCORES.append(res)
    print(SCORES)
                                                                                                                     


     none explainer_name  sparsity dataset true_label_as_target hard_mask  \
0                 random     0.000    cora                 True      True   
1                 random     0.000    cora                 True      True   
2                 random     0.000    cora                 True      True   
3                 random     0.000    cora                 True      True   
4                 random     0.000    cora                 True      True   
...   ...            ...       ...     ...                  ...       ...   
1595        pgmexplainer     0.997    cora                False     False   
1596        pgmexplainer     0.997    cora                False     False   
1597        pgmexplainer     0.997    cora                False     False   
1598        pgmexplainer     0.997    cora                False     False   
1599        pgmexplainer     0.997    cora                False     False   

     explain_graph  num_test  seed data_save_dir     time  number_of_edges 

     none explainer_name  sparsity dataset true_label_as_target hard_mask  \
0                 random     0.000  pubmed                 True      True   
1                 random     0.000  pubmed                 True      True   
2                 random     0.000  pubmed                 True      True   
3                 random     0.000  pubmed                 True      True   
4                 random     0.000  pubmed                 True      True   
...   ...            ...       ...     ...                  ...       ...   
1595        pgmexplainer     0.997  pubmed                False     False   
1596        pgmexplainer     0.997  pubmed                False     False   
1597        pgmexplainer     0.997  pubmed                False     False   
1598        pgmexplainer     0.997  pubmed                False     False   
1599        pgmexplainer     0.997  pubmed                False     False   

     explain_graph  num_test  seed data_save_dir     time  number_of_edges 

     none explainer_name  sparsity   dataset true_label_as_target hard_mask  \
0                 random     0.000  citeseer                 True      True   
1                 random     0.000  citeseer                 True      True   
2                 random     0.000  citeseer                 True      True   
3                 random     0.000  citeseer                 True      True   
4                 random     0.000  citeseer                 True      True   
...   ...            ...       ...       ...                  ...       ...   
1593        pgmexplainer     0.997  citeseer                False     False   
1594        pgmexplainer     0.997  citeseer                False     False   
1595        pgmexplainer     0.997  citeseer                False     False   
1596        pgmexplainer     0.997  citeseer                False     False   
1597        pgmexplainer     0.997  citeseer                False     False   

     explain_graph  num_test  seed data_save_dir   

     none explainer_name  sparsity  dataset true_label_as_target hard_mask  \
0                 random     0.000  cornell                 True      True   
1                 random     0.000  cornell                 True      True   
2                 random     0.000  cornell                 True      True   
3                 random     0.000  cornell                 True      True   
4                 random     0.000  cornell                 True      True   
...   ...            ...       ...      ...                  ...       ...   
1595        pgmexplainer     0.997  cornell                False     False   
1596        pgmexplainer     0.997  cornell                False     False   
1597        pgmexplainer     0.997  cornell                False     False   
1598        pgmexplainer     0.997  cornell                False     False   
1599        pgmexplainer     0.997  cornell                False     False   

     explain_graph  num_test  seed data_save_dir    time  numbe

     none explainer_name  sparsity dataset true_label_as_target hard_mask  \
0                 random     0.000   texas                 True      True   
1                 random     0.000   texas                 True      True   
2                 random     0.000   texas                 True      True   
3                 random     0.000   texas                 True      True   
4                 random     0.000   texas                 True      True   
...   ...            ...       ...     ...                  ...       ...   
1595        pgmexplainer     0.997   texas                False     False   
1596        pgmexplainer     0.997   texas                False     False   
1597        pgmexplainer     0.997   texas                False     False   
1598        pgmexplainer     0.997   texas                False     False   
1599        pgmexplainer     0.997   texas                False     False   

     explain_graph  num_test  seed data_save_dir    time  number_of_edges  

     none explainer_name  sparsity    dataset true_label_as_target hard_mask  \
0                 random     0.000  wisconsin                 True      True   
1                 random     0.000  wisconsin                 True      True   
2                 random     0.000  wisconsin                 True      True   
3                 random     0.000  wisconsin                 True      True   
4                 random     0.000  wisconsin                 True      True   
...   ...            ...       ...        ...                  ...       ...   
1591        pgmexplainer     0.997  wisconsin                False     False   
1592        pgmexplainer     0.997  wisconsin                False     False   
1593        pgmexplainer     0.997  wisconsin                False     False   
1594        pgmexplainer     0.997  wisconsin                False     False   
1595        pgmexplainer     0.997  wisconsin                False     False   

     explain_graph  num_test  seed data

     none explainer_name  sparsity dataset true_label_as_target hard_mask  \
0                 random     0.000   actor                 True      True   
1                 random     0.000   actor                 True      True   
2                 random     0.000   actor                 True      True   
3                 random     0.000   actor                 True      True   
4                 random     0.000   actor                 True      True   
...   ...            ...       ...     ...                  ...       ...   
1595        pgmexplainer     0.997   actor                False     False   
1596        pgmexplainer     0.997   actor                False     False   
1597        pgmexplainer     0.997   actor                False     False   
1598        pgmexplainer     0.997   actor                False     False   
1599        pgmexplainer     0.997   actor                False     False   

     explain_graph  num_test  seed data_save_dir     time  number_of_edges 

     none explainer_name  sparsity    dataset true_label_as_target hard_mask  \
0                 random     0.000  chameleon                 True      True   
1                 random     0.000  chameleon                 True      True   
2                 random     0.000  chameleon                 True      True   
3                 random     0.000  chameleon                 True      True   
4                 random     0.000  chameleon                 True      True   
...   ...            ...       ...        ...                  ...       ...   
1595        pgmexplainer     0.997  chameleon                False     False   
1596        pgmexplainer     0.997  chameleon                False     False   
1597        pgmexplainer     0.997  chameleon                False     False   
1598        pgmexplainer     0.997  chameleon                False     False   
1599        pgmexplainer     0.997  chameleon                False     False   

     explain_graph  num_test  seed data

     none explainer_name  sparsity   dataset true_label_as_target hard_mask  \
0                 random     0.000  squirrel                 True      True   
1                 random     0.000  squirrel                 True      True   
2                 random     0.000  squirrel                 True      True   
3                 random     0.000  squirrel                 True      True   
4                 random     0.000  squirrel                 True      True   
...   ...            ...       ...       ...                  ...       ...   
1433        pgmexplainer     0.997  squirrel                False     False   
1434        pgmexplainer     0.997  squirrel                False     False   
1435        pgmexplainer     0.997  squirrel                False     False   
1436        pgmexplainer     0.997  squirrel                False     False   
1437        pgmexplainer     0.997  squirrel                False     False   

     explain_graph  num_test  seed data_save_dir   

     none explainer_name  sparsity   dataset true_label_as_target hard_mask  \
0                 random     0.000  facebook                 True      True   
1                 random     0.000  facebook                 True      True   
2                 random     0.000  facebook                 True      True   
3                 random     0.000  facebook                 True      True   
4                 random     0.000  facebook                 True      True   
...   ...            ...       ...       ...                  ...       ...   
1595        pgmexplainer     0.997  facebook                False     False   
1596        pgmexplainer     0.997  facebook                False     False   
1597        pgmexplainer     0.997  facebook                False     False   
1598        pgmexplainer     0.997  facebook                False     False   
1599        pgmexplainer     0.997  facebook                False     False   

     explain_graph  num_test  seed data_save_dir   

In [9]:
avg_scores = pd.concat(SCORES)
avg_scores

num_test  \
dataset  true_label_as_target hard_mask sparsity explainer_name                 
cora     False                False     0.000    basic_gnnexplainer     100.0   
                                                 distance               100.0   
                                                 gnnexplainer           100.0   
                                                 gradcam                100.0   
                                                 ig                     100.0   
...                                                                       ...   
facebook True                 True      0.997    occlusion              100.0   
                                                 pagerank               100.0   
                                                 pgmexplainer           100.0   
                                                 random                 100.0   
                                                 sa                     100.0   

                                                                     fidelity_acc+  \
dataset  true_label_as_target hard_mask sparsity explainer_name                      
cora     False                False     0.000    basic_gnnexplainer         -1.000   
                                                 distance                   -1.000   
                                                 gnnexplainer               -1.000   
                                                 gradcam                    -1.000   
                                                 ig                         -1.000   
...                                                                            ...   
facebook True                 True      0.997    occlusion                   0.862   
                                                 pagerank                    0.786   
                                                 pgmexplainer                0.550   
                                                 random                      0.072   
                                                 sa                          0.265   

                                                                     fidelity_acc-  \
dataset  true_label_as_target hard_mask sparsity explainer_name                      
cora     False                False     0.000    basic_gnnexplainer         -1.000   
                                                 distance                   -1.000   
                                                 gnnexplainer               -1.000   
                                                 gradcam                    -1.000   
                                                 ig                         -1.000   
...                                                                            ...   
facebook True                 True      0.997    occlusion                   0.012   
                                                 pagerank                    0.044   
                                                 pgmexplainer                0.040   
                                                 random                      0.785   
                                                 sa                          0.550   

                                                                     fidelity_prob+  \
dataset  true_label_as_target hard_mask sparsity explainer_name                       
cora     False                False     0.000    basic_gnnexplainer       -1.000000   
                                                 distance                 -1.000000   
                                                 gnnexplainer             -1.000000   
                                                 gradcam                  -1.000000   
                                                 ig                       -1.000000   
...                                                                             ...   
facebook True                 True      0.997    occlusion                 0.7513

In [10]:
avg_scores = avg_scores.reset_index()
avg_scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_sparsity_avg10expe.csv', index=False)

### GNN scores

In [11]:
GNN_SCORES = []
for data in datasets:

    logdir = f'/cluster/home/kamara/checkpoints/node_classification/real/sparsity_real_avg10expe_{data}/logs'
    jsonpath = f'/cluster/home/kamara/Explain/configs/config_real_sparsity_avg10expe_{data}.json'
    ranges = get_param_ranges(jsonpath)
    gnn_train, gnn_test = get_gnn_info(logdir)
    
    df_gnn_train = get_df_results(gnn_train, ranges, name, metrics = ["prec", "recall", "acc"])
    df_gnn_test = get_df_results(gnn_test, ranges, name, metrics = ["prec", "recall", "acc"])
    df_gnn_train['f1_score'] = 2*df_gnn_train['recall']*df_gnn_train['prec']/(df_gnn_train['recall']+df_gnn_train['prec'])
    df_gnn_test['f1_score'] = 2*df_gnn_test['recall']*df_gnn_test['prec']/(df_gnn_test['recall']+df_gnn_test['prec'])

    df_gnn_train = df_gnn_train.groupby(by=['dataset']).mean().reset_index()
    df_gnn_test = df_gnn_test.groupby(by=['dataset']).mean().reset_index()

    gnn_scores = pd.merge(df_gnn_train, df_gnn_test, on=['dataset', 'sparsity', 'num_test'], suffixes=['_train', '_test'])
    gnn_scores = gnn_scores.drop(columns=["sparsity", "num_test", "seed_train", "seed_test"])
    print('res', gnn_scores)

    GNN_SCORES.append(gnn_scores)
    print(GNN_SCORES)

res   dataset  prec_train  recall_train  acc_train  f1_score_train  prec_test  \
0    cora         1.0           1.0        1.0             1.0   0.781341   

   recall_test  acc_test  f1_score_test  
0     0.817384    0.8035       0.798947  
[  dataset  prec_train  recall_train  acc_train  f1_score_train  prec_test  \
0    cora         1.0           1.0        1.0             1.0   0.781341   

   recall_test  acc_test  f1_score_test  
0     0.817384    0.8035       0.798947  ]
res   dataset  prec_train  recall_train  acc_train  f1_score_train  prec_test  \
0  pubmed         1.0           1.0        1.0             1.0   0.771646   

   recall_test  acc_test  f1_score_test  
0     0.781766    0.7785        0.77667  
[  dataset  prec_train  recall_train  acc_train  f1_score_train  prec_test  \
0    cora         1.0           1.0        1.0             1.0   0.781341   

   recall_test  acc_test  f1_score_test  
0     0.817384    0.8035       0.798947  ,   dataset  prec_train  recall_tr

res      dataset  prec_train  recall_train  acc_train  f1_score_train  prec_test  \
0  chameleon    0.817928      0.806324    0.80641        0.812068   0.646635   

   recall_test  acc_test  f1_score_test  
0     0.634489  0.632237       0.640485  
[  dataset  prec_train  recall_train  acc_train  f1_score_train  prec_test  \
0    cora         1.0           1.0        1.0             1.0   0.781341   

   recall_test  acc_test  f1_score_test  
0     0.817384    0.8035       0.798947  ,   dataset  prec_train  recall_train  acc_train  f1_score_train  prec_test  \
0  pubmed         1.0           1.0        1.0             1.0   0.771646   

   recall_test  acc_test  f1_score_test  
0     0.781766    0.7785        0.77667  ,     dataset  prec_train  recall_train  acc_train  f1_score_train  prec_test  \
0  citeseer         1.0           1.0        1.0             1.0   0.651307   

   recall_test  acc_test  f1_score_test  
0     0.652012    0.6764       0.651656  ,    dataset  prec_train  re

In [12]:
avg_gnn_scores = pd.concat(GNN_SCORES)
avg_gnn_scores

,dataset,prec_train,recall_train,acc_train,f1_score_train,prec_test,recall_test,acc_test,f1_score_test
0,cora,1.000000,1.000000,1.000000,1.000000,0.781341,0.817384,0.803500,0.798947
0,pubmed,1.000000,1.000000,1.000000,1.000000,0.771646,0.781766,0.778500,0.776670
0,citeseer,1.000000,1.000000,1.000000,1.000000,0.651307,0.652012,0.676400,0.651656
0,cornell,0.927224,0.908135,0.974713,0.917484,0.354914,0.338861,0.532432,0.344153
0,texas,0.972625,0.966848,0.991954,0.969666,0.296897,0.277057,0.510811,0.284965
0,wisconsin,0.980174,0.955045,0.975833,0.967295,0.397882,0.406152,0.535294,0.396670
0,actor,0.594783,0.480918,0.538185,0.531375,0.260683,0.249017,0.285921,0.253762
0,chameleon,0.817928,0.806324,0.806410,0.812068,0.646635,0.634489,0.632237,0.640485
0,squirrel,0.469282,0.474848,0.475527,0.467952,0.333126,0.373366,0.376114,0.350042
0,facebook,0.931204,0.926199,0.933335,0.928694,0.923283,0.917607,0.926346,0.920435


In [13]:
#gnn_scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_gnn_scores.csv', index=False)

## Node Classification - Synthetic

In [6]:
def get_info_syn(logdir):
    infos, initial_mask_infos, transformed_mask_infos, accuracy, fidelity = {}, {}, {}, {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__infos:')
            _, initial_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__initial_edge_mask_infos:')
            _, transformed_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__transformed_mask_infos:')
            _, fidelity[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__fidelity:')             
            _, accuracy[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__accuracy:')             
    return(infos, initial_mask_infos, transformed_mask_infos, accuracy, fidelity)



### Topk

In [7]:
logdir = f'/cluster/work/zhang/kamara/checkpoints/node_classification/syn/topk/topk_syn_5/logs'
jsonpath = f'/cluster/home/kamara/Explain/configs/syn/topk/config_syn_topk_5expe.json'
infos, initial_mask_infos, transformed_mask_infos, accuracy, fidelity = get_info_syn(logdir)
gnn_train, gnn_test = get_gnn_info(logdir)

print(list(fidelity.keys())[1])
ranges = get_param_ranges(jsonpath)

_explainer_name=sa_dataset=syn6_true_label_as_target=True_hard_mask=True_seed=0


In [8]:
#name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_dataset={dataset}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}'
name = '{none}_explainer_name={explainer_name}_dataset={dataset}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}_seed={seed}'


#### GNN scores

In [9]:
df_gnn_train = get_df_results(gnn_train, ranges, name, metrics = ["prec", "recall", "f1_score", "acc"])
df_gnn_test = get_df_results(gnn_test, ranges, name, metrics = ["prec", "recall", "f1_score", "acc"])

df_gnn_train = df_gnn_train.groupby(by=['dataset']).mean().reset_index()
df_gnn_test = df_gnn_test.groupby(by=['dataset']).mean().reset_index()
df_gnn_train['f1_score'] = 2*df_gnn_train['recall']*df_gnn_train['prec']/(df_gnn_train['recall']+df_gnn_train['prec'])
df_gnn_test['f1_score'] = 2*df_gnn_test['recall']*df_gnn_test['prec']/(df_gnn_test['recall']+df_gnn_test['prec'])


gnn_scores = pd.merge(df_gnn_train, df_gnn_test, on=['dataset', 'num_test'], suffixes=['_train', '_test'])
gnn_scores = gnn_scores.drop(columns=["num_test", "seed_train", "seed_test"])
gnn_scores

,dataset,prec_train,recall_train,f1_score_train,acc_train,prec_test,recall_test,f1_score_test,acc_test
0,syn1,0.971151,0.976283,0.973710,0.979439,0.972222,0.979167,0.975682,0.985714
1,syn3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,syn4,0.986120,0.983051,0.984583,0.984962,1.000000,1.000000,1.000000,1.000000
3,syn5,0.902598,0.884369,0.893390,0.896809,0.925287,0.870000,0.896792,0.895161
4,syn6,0.988189,0.987705,0.987947,0.988785,1.000000,1.000000,1.000000,1.000000


In [29]:
#gnn_scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_gnn_scores.csv', index=False)

#### Explainers

In [9]:
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges"])
df_initial_mask_infos = get_df_results(initial_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg"])
df_tranformed_mask_infos = get_df_results(transformed_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg", "cc_ratio", "topk"], selection="all")
df_acc = get_df_results(accuracy, ranges, name, metrics = ["f1_score", "precision", "recall", "topk"], selection="all")
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 
                                                           'fidelity_gnn_acc+', 'fidelity_gnn_acc-', 'fidelity_gnn_prob+', 'fidelity_gnn_prob-', 'topk'], selection="all")

In [10]:
df_mask_infos = pd.merge(df_initial_mask_infos, df_tranformed_mask_infos, suffixes=['_init', '_transf'], on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask"])
scores = pd.merge(df_fid, df_acc, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask", "topk"])
scores = pd.merge(scores, df_mask_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask", "topk"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask"])
scores
#scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_sparsity_all.csv', index=False)


,none_x,explainer_name,params_list_x,dataset,true_label_as_target,hard_mask,explain_graph_x,num_test,seed,data_save_dir,...,explain_graph_transf,mask_size_transf,mask_entropy_transf,max_avg_transf,cc_ratio,none,params_list,explain_graph,time,number_of_edges
0,,random,"1,5,10,15,20,25,50,100",syn1,True,True,False,100,0,data,...,False,1.00,0.000000,1.000000,0.500000,,"1,5,10,15,20,25,50,100",False,0.0001,4110
1,,random,"1,5,10,15,20,25,50,100",syn1,True,True,False,100,0,data,...,False,5.00,1.609438,0.999137,0.492778,,"1,5,10,15,20,25,50,100",False,0.0001,4110
2,,random,"1,5,10,15,20,25,50,100",syn1,True,True,False,100,0,data,...,False,10.00,2.302585,0.998306,0.484598,,"1,5,10,15,20,25,50,100",False,0.0001,4110
3,,random,"1,5,10,15,20,25,50,100",syn1,True,True,False,100,0,data,...,False,15.00,2.708050,0.997599,0.478049,,"1,5,10,15,20,25,50,100",False,0.0001,4110
4,,random,"1,5,10,15,20,25,50,100",syn1,True,True,False,100,0,data,...,False,20.00,2.995731,0.997061,0.469671,,"1,5,10,15,20,25,50,100",False,0.0001,4110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,,subgraphx,"1,5,10,15,20,25,50,100",syn6,False,False,False,100,3,data,...,False,6.36,1.745893,1.000000,0.277167,,"1,5,10,15,20,25,50,100",False,64.2504,3948
8188,,subgraphx,"1,5,10,15,20,25,50,100",syn6,False,False,False,100,3,data,...,False,6.36,1.745893,1.000000,0.277167,,"1,5,10,15,20,25,50,100",False,64.2504,3948
8189,,subgraphx,"1,5,10,15,20,25,50,100",syn6,False,False,False,100,3,data,...,False,6.36,1.745893,1.000000,0.277167,,"1,5,10,15,20,25,50,100",False,64.2504,3948
8190,,subgraphx,"1,5,10,15,20,25,50,100",syn6,False,False,False,100,3,data,...,False,6.36,1.745893,1.000000,0.277167,,"1,5,10,15,20,25,50,100",False,64.2504,3948


In [11]:
res = scores.groupby(by=['dataset', "true_label_as_target", "hard_mask", 'topk', 'explainer_name']).mean()
res = res.drop(columns=['seed'])
#res[['mask_size', "mask_entropy", "max_avg"]]
res

num_test  \
dataset true_label_as_target hard_mask topk  explainer_name                 
syn1    False                False     1.0   basic_gnnexplainer     100.0   
                                             distance               100.0   
                                             gnnexplainer           100.0   
                                             ig                     100.0   
                                             occlusion              100.0   
...                                                                   ...   
syn6    True                 True      100.0 pgexplainer            100.0   
                                             pgmexplainer           100.0   
                                             random                 100.0   
                                             sa                     100.0   
                                             subgraphx              100.0   

                                                                 fidelity_acc+  \
dataset true_label_as_target hard_mask topk  explainer_name                      
syn1    False                False     1.0   basic_gnnexplainer         -1.000   
                                             distance                   -1.000   
                                             gnnexplainer               -1.000   
                                             ig                         -1.000   
                                             occlusion                  -1.000   
...                                                                        ...   
syn6    True                 True      100.0 pgexplainer                 0.238   
                                             pgmexplainer                0.616   
                                             random                      0.014   
                                             sa                          0.280   
                                             subgraphx                   0.630   

                                                                 fidelity_acc-  \
dataset true_label_as_target hard_mask topk  explainer_name                      
syn1    False                False     1.0   basic_gnnexplainer         -1.000   
                                             distance                   -1.000   
                                             gnnexplainer               -1.000   
                                             ig                         -1.000   
                                             occlusion                  -1.000   
...                                                                        ...   
syn6    True                 True      100.0 pgexplainer                 0.526   
                                             pgmexplainer                0.058   
                                             random                      0.618   
                                             sa                          0.614   
                                             subgraphx                   0.330   

                                                                 fidelity_prob+  \
dataset true_label_as_target hard_mask topk  explainer_name                       
syn1    False                False     1.0   basic_gnnexplainer       -1.000000   
                                             distance                 -1.000000   
                                             gnnexplainer             -1.000000   
                                             ig                       -1.000000   
                                             occlusion                -1.000000   
...                                                                         ...   
syn6    True                 True      100.0 pgexplainer               0.204853   
                                             pgmexplainer              0.510130   
                                             random                    0.008127   
                

In [13]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/syn/topk/nc_syn_topk_5expe.csv', index=False)

### Mask property (size, entropy, avg max value, connected components)

In [15]:
logdir = f'/cluster/work/zhang/kamara/checkpoints/node_classification/syn/topk/topk_syn_cc/logs'
jsonpath = f'/cluster/home/kamara/Explain/configs/syn/topk/config_syn_topk_cc.json'
infos, initial_mask_infos, transformed_mask_infos, accuracy, fidelity = get_info_syn(logdir)
gnn_train, gnn_test = get_gnn_info(logdir)

print(list(fidelity.keys())[1])
ranges = get_param_ranges(jsonpath)

_explainer_name=occlusion


In [16]:
name = '{none}_explainer_name={explainer_name}'

In [17]:
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges"])
df_initial_mask_infos = get_df_results(initial_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg", "cc_ratio"])
df_tranformed_mask_infos = get_df_results(transformed_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg", "cc_ratio", "topk"], selection="all")
df_acc = get_df_results(accuracy, ranges, name, metrics = ["f1_score", "precision", "recall", "topk"], selection="all")
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 
                                                           'fidelity_gnn_acc+', 'fidelity_gnn_acc-', 'fidelity_gnn_prob+', 'fidelity_gnn_prob-', 'topk'], selection="all")

In [18]:
df_mask_infos = pd.merge(df_initial_mask_infos, df_tranformed_mask_infos, suffixes=['_init', '_transf'], on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask"])
scores = pd.merge(df_fid, df_acc, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask", "topk"])
scores = pd.merge(scores, df_mask_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask", "topk"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "true_label_as_target", "hard_mask"])
scores
#scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_sparsity_all.csv', index=False)


,none_x,explainer_name,params_list_x,dataset,true_label_as_target,hard_mask,explain_graph_x,num_test,seed,data_save_dir,...,explain_graph_transf,mask_size_transf,mask_entropy_transf,max_avg_transf,cc_ratio_transf,none,params_list,explain_graph,time,number_of_edges
0,,random,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",syn1,True,True,False,20,0,data,...,False,1.0,0.000000,1.000000,0.500000,,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",False,0.0002,4110
1,,random,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",syn1,True,True,False,20,0,data,...,False,2.0,0.693147,0.999795,0.500000,,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",False,0.0002,4110
2,,random,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",syn1,True,True,False,20,0,data,...,False,3.0,1.098612,0.999626,0.495000,,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",False,0.0002,4110
3,,random,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",syn1,True,True,False,20,0,data,...,False,4.0,1.386294,0.999397,0.492857,,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",False,0.0002,4110
4,,random,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",syn1,True,True,False,20,0,data,...,False,5.0,1.609438,0.999178,0.494444,,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",False,0.0002,4110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,,subgraphx,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",syn1,True,True,False,20,0,data,...,False,9.6,1.990807,1.000000,0.276667,,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",False,0.1252,4110
161,,subgraphx,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",syn1,True,True,False,20,0,data,...,False,9.6,1.990807,1.000000,0.276667,,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",False,0.1252,4110
162,,subgraphx,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",syn1,True,True,False,20,0,data,...,False,9.6,1.990807,1.000000,0.276667,,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",False,0.1252,4110
163,,subgraphx,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",syn1,True,True,False,20,0,data,...,False,9.6,1.990807,1.000000,0.276667,,"1,2,3,4,5,6,7,8,9,10,15,20,25,50,100",False,0.1252,4110


In [19]:
res = scores.groupby(by=['dataset', "true_label_as_target", "hard_mask", 'topk', 'explainer_name']).mean()
res = res.drop(columns=['seed'])
#res[['mask_size', "mask_entropy", "max_avg"]]
res

num_test  \
dataset true_label_as_target hard_mask topk  explainer_name                 
syn1    True                 True      1.0   basic_gnnexplainer      20.0   
                                             distance                20.0   
                                             gnnexplainer            20.0   
                                             ig                      20.0   
                                             occlusion               20.0   
...                                                                   ...   
                                       100.0 pgexplainer             20.0   
                                             pgmexplainer            20.0   
                                             random                  20.0   
                                             sa                      20.0   
                                             subgraphx               20.0   

                                                                 fidelity_acc+  \
dataset true_label_as_target hard_mask topk  explainer_name                      
syn1    True                 True      1.0   basic_gnnexplainer           0.05   
                                             distance                     0.10   
                                             gnnexplainer                 0.05   
                                             ig                           0.10   
                                             occlusion                    0.45   
...                                                                        ...   
                                       100.0 pgexplainer                  0.25   
                                             pgmexplainer                 0.45   
                                             random                       0.05   
                                             sa                           0.55   
                                             subgraphx                    0.30   

                                                                 fidelity_acc-  \
dataset true_label_as_target hard_mask topk  explainer_name                      
syn1    True                 True      1.0   basic_gnnexplainer           0.55   
                                             distance                     0.55   
                                             gnnexplainer                 0.55   
                                             ig                           0.55   
                                             occlusion                    0.55   
...                                                                        ...   
                                       100.0 pgexplainer                  0.55   
                                             pgmexplainer                 0.15   
                                             random                       0.55   
                                             sa                           0.55   
                                             subgraphx                    0.40   

                                                                 fidelity_prob+  \
dataset true_label_as_target hard_mask topk  explainer_name                       
syn1    True                 True      1.0   basic_gnnexplainer        0.021082   
                                             distance                 -0.024823   
                                             gnnexplainer             -0.004908   
                                             ig                        0.002297   
                                             occlusion                 0.326566   
...                                                                         ...   
                                       100.0 pgexplainer               0.122988   
                                             pgmexplainer              0.363850   
                                             random                    0.006008   
                

In [20]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/syn/topk/nc_syn_topk_mask_properties.csv', index=False)

### Accuracy top

In [10]:
def get_info_top(logdir):
    infos, initial_mask_infos, accuracy_top = {}, {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__infos:')
            _, initial_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__initial_edge_mask_infos:')
            _, accuracy_top[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__accuracy_top:')
    return(infos, initial_mask_infos, accuracy_top)


In [12]:
logdir = f'/cluster/work/zhang/kamara/checkpoints/node_classification/syn/topk/top_edges/logs'
jsonpath = f'/cluster/home/kamara/Explain/configs/syn/topk/config_top_edges.json'
infos, initial_mask_infos, accuracy_top = get_info_top(logdir)

ranges = get_param_ranges(jsonpath)
print(list(accuracy_top.keys())[1])

Unable to replace NaNs
__infos:{"dataset": "syn5", "explainer": "subgraphx", "number_of_edges": 3410, "num_test": 20, "num_test_final": 20, "groundtruth target": "True", "time": 4.651, "edge_mask_sparsity_init": 0.9968035190615836, "edge_mask_size_init": 10.9, "edge_mask_connected_init": null}

Unable to replace NaNs
__initial_edge_mask_infos:{"mask_size": 10.9, "mask_entropy": 2.4763032054249767, "max_avg": 1.0, "cc_ratio": null}

Unable to replace NaNs
__infos:{"dataset": "syn5", "explainer": "subgraphx", "number_of_edges": 3410, "num_test": 20, "num_test_final": 20, "groundtruth target": "True", "time": 5.6368, "edge_mask_sparsity_init": 0.99524926686217, "edge_mask_size_init": 16.2, "edge_mask_connected_init": null}

Unable to replace NaNs
__initial_edge_mask_infos:{"mask_size": 16.2, "mask_entropy": 2.697263310621022, "max_avg": 1.0, "cc_ratio": null}

Unable to replace NaNs
__infos:{"dataset": "syn5", "explainer": "subgraphx", "number_of_edges": 3410, "num_test": 20, "num_test_fi

In [13]:
name = '{none}_explainer_name={explainer_name}_dataset={dataset}_seed={seed}'

In [14]:
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges"])
df_initial_mask_infos = get_df_results(initial_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg", "cc_ratio"])
df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision'])

In [16]:
scores = df_acc_top
scores = pd.merge(scores, df_initial_mask_infos, on=["dataset", "explainer_name", "data_save_dir", "seed"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "data_save_dir", "seed"])
scores

,none_x,explainer_name,dataset,top_acc_x,seed,num_test_x,data_save_dir,f1_score,recall,precision,...,num_test_y,mask_size,mask_entropy,max_avg,cc_ratio,none,top_acc,num_test,time,number_of_edges
0,,random,syn1,True,0,20,data,0.000000,0.000000,0.000000,...,20,4104.70,8.126336,0.426719,0.001429,,True,20,0.0002,4110
1,,random,syn1,True,1,20,data,0.008333,0.008333,0.008333,...,20,4104.55,8.126173,0.504091,0.001429,,True,20,0.0002,4110
2,,random,syn1,True,2,20,data,0.008333,0.008333,0.008333,...,20,4105.15,8.126489,0.508101,0.001429,,True,20,0.0003,4110
3,,random,syn1,True,3,20,data,0.000000,0.000000,0.000000,...,20,4104.80,8.127410,0.483644,0.001429,,True,20,0.0002,4110
4,,random,syn1,True,4,20,data,0.008333,0.008333,0.008333,...,20,4104.10,8.126385,0.496112,0.001429,,True,20,0.0002,4110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,,subgraphx,syn4,True,3,20,data,0.251364,0.208333,0.327500,...,20,8.60,2.124361,1.000000,0.195833,,True,20,4.9707,1950
256,,subgraphx,syn5,True,3,20,data,0.238430,0.200000,0.298683,...,20,11.80,2.122123,1.000000,0.234311,,True,20,3.9769,3410
257,,subgraphx,syn6,True,2,20,data,0.627778,0.550000,0.875000,...,20,6.80,1.730496,1.000000,0.290000,,True,20,273.4840,3948
258,,subgraphx,syn6,True,3,20,data,0.583333,0.490000,0.820833,...,20,6.20,1.709424,1.000000,0.288333,,True,20,54.9894,3948


In [17]:
res = scores.groupby(by=['dataset', 'explainer_name']).mean()
res

seed  num_test_x  f1_score    recall  precision  \
dataset explainer_name                                                        
syn1    basic_gnnexplainer   2.0        20.0  0.309697  0.303333   0.317333   
        distance             2.0        20.0  0.678455  0.668333   0.691000   
        gnnexplainer         2.0        20.0  0.297576  0.291667   0.304667   
        ig                   2.0        20.0  0.446576  0.443333   0.450667   
        occlusion            2.0        20.0  0.171667  0.171667   0.171667   
        pagerank             2.0        20.0  0.861667  0.861667   0.861667   
        pgexplainer          2.0        20.0  0.017000  0.016667   0.017500   
        pgmexplainer         2.0        20.0  0.543333  0.543333   0.543333   
        random               2.0        20.0  0.005000  0.005000   0.005000   
        sa                   2.0        20.0  0.043333  0.043333   0.043333   
        subgraphx            2.0        20.0  0.612544  0.522222   0.822500   
syn3    basic_gnnexplainer   2.0        20.0  0.663819  0.620833   0.717982   
        distance             2.0        20.0  0.730820  0.720833   0.742045   
        gnnexplainer         2.0        20.0  0.660332  0.614167   0.718730   
        ig                   2.0        20.0  0.185991  0.184167   0.188106   
        occlusion            2.0        20.0  0.000000  0.000000   0.000000   
        pagerank             2.0        20.0  0.937879  0.936667   0.939333   
        pgexplainer          2.0        20.0  0.011087  0.010833   0.011364   
        pgmexplainer         2.0        20.0  0.663484  0.659167   0.669045   
        random               2.0        20.0  0.006667  0.006667   0.006667   
        sa                   2.0        20.0  0.000000  0.000000   0.000000   
        subgraphx            3.0        20.0  0.758212  0.679167   0.891439   
syn4    basic_gnnexplainer   2.0        20.0  0.233333  0.226667   0.241333   
        distance             2.0        20.0  0.250758  0.246667   0.255667   
        gnnexplainer         2.0        20.0  0.239333  0.233333   0.246833   
        ig                   2.0        20.0  0.244303  0.238333   0.252667   
        occlusion            2.0        20.0  0.220000  0.220000   0.220000   
        pagerank             2.0        20.0  0.323333  0.323333   0.323333   
        pgexplainer          2.0        20.0  0.000000  0.000000   0.000000   
        pgmexplainer         2.0        20.0  0.350121  0.345000   0.356667   
        random               2.0        20.0  0.003333  0.003333   0.003333   
        sa                   2.0        20.0  0.071303  0.070000   0.073167   
        subgraphx            1.5        20.0  0.217500  0.183333   0.273750   
syn5    basic_gnnexplainer   2.0        20.0  0.327812  0.310833   0.348487   
        distance             2.0        20.0  0.385389  0.378333   0.393424   
        gnnexplainer         2.0        20.0  0.349876  0.335000   0.367475   
        ig                   2.0        20.0  0.435313  0.434167   0.436621   
        occlusion            2.0        20.0  0.268015  0.259167   0.278864   
        pagerank             2.0        20.0  0.485000  0.485000   0.485000   
        pgexplainer          2.0        20.0  0.000000  0.000000   0.000000   
        pgmexplainer         2.0        20.0  0.473294  0.470000   0.476970   
        random               2.0        20.0  0.000833  0.000833   0.000833   
        sa                   2.0        20.0  0.221667  0.221667   0.221667   
        subgraphx            3.0        20.0  0.238430  0.200000   0.298683   
syn6    basic_gnnexplainer   2.0        20.0  0.126889  0.126000   0.128000   
        distance             2.0        20.0  0.733333  0.732000   0.735000   
        gnnexplainer         2.0        20.0  0.172222  0.172000   0.172500   
        ig                   2.0        20.0  0.399778  0.382000   0.422000   
        occlusion            2.0        20.0  0.074000  0.074000   0.074000   

In [19]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/syn/topk/nc_syn_top_edges.csv', index=False)

### Gc Layers

In [50]:
expe = "gc_layers"
logdir = f'/cluster/home/kamara/checkpoints/{expe}/logs'
jsonpath = f'/cluster/home/kamara/Explain/config_{expe}.json'
infos, accuracy, fidelity = get_info(logdir)
ranges = get_param_ranges(jsonpath)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_num_gc_layers={num_gc_layers}'


_explainer_name=pagerank_num_gc_layers=1


In [51]:
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-'])
scores = pd.merge(df_acc, df_fid, on=["dataset", "num_gc_layers", "explainer_name", "num_test_nodes", "data_save_dir"])
scores = scores.groupby(by=['num_gc_layers', 'explainer_name']).mean()
scores

num_test_nodes  f1_score    recall  precision  \
num_gc_layers explainer_name                                                  
2             gnnexplainer             200.0  0.682879  0.652500   0.720833   
              pagerank                 200.0  0.864167  0.864167   0.864167   
3             gnnexplainer             200.0  0.329136  0.323333   0.336250   
              pagerank                 200.0  0.864167  0.864167   0.864167   
4             gnnexplainer             200.0  0.055833  0.055833   0.055833   
              pagerank                 200.0  0.860000  0.860000   0.860000   
5             gnnexplainer             200.0  0.006667  0.006667   0.006667   
              pagerank                 200.0  0.859167  0.859167   0.859167   
6             gnnexplainer             200.0  0.003333  0.003333   0.003333   
              pagerank                 200.0  0.859167  0.859167   0.859167   

                                 ged       auc  fidelity_acc+  fidelity_acc-  \
num_gc_layers explainer_name                                                   
2             gnnexplainer     4.065  0.998892          0.835          0.000   
              pagerank         2.435  0.999885          0.835          0.000   
3             gnnexplainer     8.160  0.972214          0.610          0.015   
              pagerank         2.430  0.999885          0.610          0.000   
4             gnnexplainer    12.475  0.834369          0.750          0.290   
              pagerank         2.515  0.999885          0.830          0.000   
5             gnnexplainer    14.085  0.656783          0.550          0.575   
              pagerank         2.525  0.999883          0.640          0.005   
6             gnnexplainer    14.345  0.575590          0.610          0.645   
              pagerank         2.525  0.999883          0.640          0.005   

                              fidelity_prob+  fidelity_prob-  
num_gc_layers explainer_name                                  
2             gnnexplainer          0.650817        0.000000  
              pagerank              0.650817        0.000000  
3             gnnexplainer          0.582833        0.014268  
              pagerank              0.589879       -0.000021  
4             gnnexplainer          0.571764        0.246716  
              pagerank              0.640806        0.005695  
5             gnnexplainer          0.512901        0.548674  
              pagerank              0.608210       -0.001958  
6             gnnexplainer          0.545983        0.581469  
              pagerank              0.579183       -0.000794

## Graph Classification

### Sparsity

In [13]:
logdir = f'/cluster/home/kamara/checkpoints/graph_classification/sparsity_mask/logs'
jsonpath = f'/cluster/home/kamara/checkpoints/graph_classification/sparsity_mask/sweep.json'
infos, accuracy, fidelity = get_info(logdir)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_hard_mask={hard_mask}'

ranges = get_param_ranges(jsonpath)
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges", "mask_sparsity_init", "non_zero_values_init"])
#df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 'mask_sparsity', 'expl_edges'])

#scores = pd.concat([df_acc, df_fid], axis=1, join="inner")
#scores = pd.merge(df_acc_top, df_acc, suffixes=['_top', ''], on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
scores = df_acc
print(scores)
scores = pd.merge(scores, df_fid, on=["explainer_name", "num_test", "data_save_dir", "sparsity", "hard_mask"])
scores = pd.merge(scores, df_infos, on=["explainer_name", "num_test", "data_save_dir", "sparsity", "hard_mask"])
scores


_explainer_name=random_sparsity=0.7_hard_mask=True
Empty DataFrame
Columns: []
Index: []


KeyError: 'explainer_name'

In [7]:
res = scores.groupby(by=['dataset', 'sparsity', 'explainer_name', 'hard_mask']).mean()
res

num_test  f1_score    recall  \
dataset sparsity explainer_name hard_mask                                 
syn1    0.000    distance       False         100.0  0.005823  1.000000   
                                True          100.0  0.005823  1.000000   
                 gnnexplainer   False         100.0  0.444157  1.000000   
                                True          100.0  0.444157  1.000000   
                 ig             False         100.0  0.005099  0.873333   
...                                             ...       ...       ...   
syn6    0.997    random         True          100.0  0.002500  0.004000   
                 sa             False         100.0  0.000000  0.000000   
                                True          100.0  0.000000  0.000000   
                 subgraphx      False         100.0  0.661056  0.634000   
                                True          100.0  0.661056  0.634000   

                                           precision  fidelity_acc+  \
dataset sparsity explainer_name hard_mask                             
syn1    0.000    distance       False       0.002920           0.28   
                                True        0.002920           0.76   
                 gnnexplainer   False       0.328940           0.56   
                                True        0.328940           0.76   
                 ig             False       0.002557           0.35   
...                                              ...            ...   
syn6    0.997    random         True        0.001818           0.00   
                 sa             False       0.000000           0.00   
                                True        0.000000           0.00   
                 subgraphx      False       0.700500           0.54   
                                True        0.700500           0.54   

                                           fidelity_acc-  fidelity_prob+  \
dataset sparsity explainer_name hard_mask                                  
syn1    0.000    distance       False               0.64        0.102925   
                                True                0.00        0.511743   
                 gnnexplainer   False               0.56        0.296190   
                                True                0.00        0.511743   
                 ig             False               0.64        0.042806   
...                                                  ...             ...   
syn6    0.997    random         True                0.64       -0.000110   
                 sa             False               0.64        0.001296   
                                True                0.64        0.001296   
                 subgraphx      False               0.20        0.493473   
                                True                0.20        0.493473   

                                           fidelity_prob-  mask_sparsity  \
dataset sparsity explainer_name hard_mask                                  
syn1    0.000    distance       False            0.358078       0.010139   
                                True             0.000000       0.010139   
                 gnnexplainer   False            0.348878       0.953431   
                                True             0.000000       0.953431   
                 ig             False            0.412742       0.002487   
...                                                   ...            ...   
syn6    0.997    random         True             0.599028       0.997214   
                 sa             False            0.599028       0.997214   
                                True             0.599028       0.997214   
                 subgraphx      False            0.183070       0.997829   
                                True             0.183070       0.997829   

                                           expl_edges    time  \
dataset sparsity explainer_name hard_mask                       
syn1    0.000    distance       Fal

In [8]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/sparsity_mask_nc.csv', index=False)